## NLP Tast 중 하나인 Semantic texual similarity(STS) 활용
- 유저(A)가 작성한 소개글로 **문장 유사도**를 파악하여 타 유저들이 작성한 소개글 중 가장 유사한 글을 작성한 타 유저(B)를 찾아내보자
- 카카오브레인의 Pororo 사용

### `1. 필요 라이브러리 import`

In [1]:
import pandas as pd
import numpy as np
import torch

import urllib.request

from pororo import Pororo
from sentence_transformers import SentenceTransformer

In [2]:
from tqdm import tqdm
tqdm.pandas()

In [3]:
data = pd.read_csv('user_Introduction.csv')
data

,user_id,Sex,Age,location,Introduction
0,0,여성,40대,b,좋은 인연을 만나기가 참 힘드네요 서로 배려해주고 아껴주는 친구처럼 편안한 사람...
1,1,남성,40대,k,중소기업이지만 동일업계에서 인정받는 회사에 임원입니다. 안정적인 직장에 안정적인 ...
2,2,남성,30대,b,안녕하세요. 저는 부천 있는 중소기업 현장에서 근무하고 있습니다. 나이가 있는 관...
3,3,남성,40대,e,저는 개발자입니다. 저랑 남아있는 생을 행복하게 함께보낼 배우자를 찾습니다.
4,4,여성,20대,h,🌼🌻🌼 여자 입니다 🌻🌼🌻dd
...,...,...,...,...,...
3191,3191,남성,40대,h,저는 이것저것? 장사를 하고 이 어려운 시기 잘? 버티고 있습니다ㅜㅜ 그쪽도 그런가...
3192,3192,남성,50대,h,다정다감하며 늘 긍정적이며 후회없는 삶을 위해 노력하며 생활합니다. 서로에게 위로가...
3193,3193,남성,40대,o,"안녕하세요~ 천안에 살고 있는 이해심, 배려심 많은 순수남입니다. 앞만 보고 달려오..."
3194,3194,남성,40대,h,안녕하세요 평범한 남자 입니다 좋은 인연 되었으면 줗겠습니다 비흡연 이구요 친구같은...


- `user_id` : 회원 번호
- `Sex` : 성별
- `Age` : 나이대
- `location` : 지역
- `Introudction` : 소개글

In [4]:
%%time
model = Pororo(task="sentence_embedding", lang="ko")

CPU times: user 3.39 s, sys: 1.04 s, total: 4.43 s
Wall time: 4.08 s


### `2. 기본 전처리`

In [5]:
import re

In [6]:
def basic_preprocessing(df,col):
    df.drop(df[df[col].isnull()].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    print('Isnull?', df[col].isnull().sum())
    
    # 공백 제거
    df[col] = df[col].apply(lambda x: x.strip())
    
    # 특수문자, 단모음/단자음 제거
    df[col] = df[col].apply(lambda x: re.sub(r'[\n\r]', '',x))
    df[col] = df[col].apply(lambda x: re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',x))
    df[col] = df[col].apply(lambda x: re.sub('[a-zA-Z]','',x))
    df[col] = df[col].apply(lambda x: re.sub('[-=+,#/\?:^.@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]','',x))    
    
    # utf-8 encode
    df[col] = df[col].apply(lambda x: x.encode('utf-8','ignore').decode('utf-8'))
    
    # 이모티콘 (이모지) 제거
    def del_emoji (text):
        only_BMP_pattern = re.compile("["
        u"\U00010000-\U0010FFFF"  #BMP characters 이외
                           "]+", flags=re.UNICODE)
        return only_BMP_pattern.sub(r'', text)
    
    df[col] = df[col].apply(del_emoji)

    return df

In [7]:
data = basic_preprocessing(data, 'Introduction')

Isnull? 0


In [8]:
display(data[data['Introduction']==''])
data.drop(data[data['Introduction']==''].index, inplace=True)

,user_id,Sex,Age,location,Introduction
130,130,여성,20대,h,
185,186,남성,20대,h,
250,251,남성,100대,f,
1584,1586,남성,40대,k,
2371,2373,여성,40대,k,


### `3. 유사도 추출`

     성별에 따라 유사한 회원 3명을 보여줌 (남성회원에게는 여성회원, 여성회원에게는 남성회원)
     *위의 경우에는 같은 나이대 회원을 보여줌*

### **`Case 2`**

In [9]:
df = data.copy()

In [10]:
df.loc[0,'Sex']

'여성'

`(1) 성별, 나이 split`

In [11]:
class generator:
    
    def __init__(self,df):
        self.df = df

    def split_sex(self,user_df):
        if user_df.loc[0,'Sex']=='여성':
            target_df= self.df[self.df['Sex']=='남성'].reset_index(drop=True)
        elif user_df.loc[0,'Sex'] =='남성':
            target_df = self.df[self.df['Sex']=='여성'].reset_index(drop=True)
        else:
            raise Exception('Invalid mem_sex value')

        return target_df
    
    def split_age(self,user_df, target_df):
        
        user_age = user_df.loc[0,'Age']
        
        if user_df.loc[0,'Sex'] =='여성':
            target_df = target_df[target_df['Age']==user_age]  
        elif user_df.loc[0,'Sex'] =='남성':
            target_df = target_df[target_df['Age']==user_age]
        else:
            raise Exception('Invalid Age value')
    
        return target_df

`(2) similar embedding`

In [12]:
from sentence_transformers import util

class get_similar_user(generator):
     
    def __init__(self, df):
        self.df = df
               
    def return_df(self,user_df):
        
        def cos_sim(A, B):
            return round(dot(A, B)/(norm(A)*norm(B)), 3)
        
        def calc_similarity(query_embedding, corpus_embeddings):
            cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
            cos_scores = cos_scores.cpu()
            cos_scores = cos_scores.detach().numpy().tolist()[0]
            cos_scores = round(cos_scores, 3)
            return cos_scores    
        

        target_df = self.split_sex(user_df)
        target_df = self.split_age(user_df=user_df,target_df=target_df)
        
        # Introduction embedding 
        target_df['mate_embedding'] = target_df['Introduction'].progress_map(model)
        user_emb = model(user_df['Introduction'][0])  
        target_df['similar_score'] = target_df['mate_embedding'].progress_map(lambda x: calc_similarity(user_emb, x))
        
        # 상위 3명 Top-3뽑기
        target_df.sort_values(by ='similar_score', ascending=False, inplace=True)
        target_df.drop(columns='mate_embedding', inplace=True)
        result = target_df[:3].reset_index(drop=True)
        
        
        return result

`(3) 조건 거르기`

In [13]:
class check_condition(get_similar_user):
    
    def __init__(self, df, user_df):
        self.df = df
        self.user_df = user_df
        self.target_df = self.return_df(user_df)
        
    def loc_condition(self):
        user_loc = self.user_df['location'][0]
        loc_sim_df = self.target_df[self.target_df['location']==user_loc].reset_index(drop=True)
        
        return loc_sim_df

### ``main``

In [14]:
class final_get_user(check_condition):
    
    def __init__(self,df):
        self.df = df
    
    def get_final_user(self):
        
        get_final_df = pd.DataFrame()
        
        for i in range(self.df.shape[0]):
            user_df = self.df.iloc[i].to_frame().transpose().reset_index(drop=True)
            target_df = self.return_df(user_df)
            score = target_df['similar_score']
            target_df.drop(columns='similar_score', inplace=True)
            target_df.columns = ['ptr_'+col for col in user_df.columns]
            
            result = pd.concat([user_df,target_df])
            result['score'] = score
            result.fillna(method='ffill')
            
            get_final_df = pd.concat([get_final_df, result])
            
        return get_final_df

In [15]:
fgu = final_get_user(df)

In [16]:
result_df = fgu.get_final_user()
result_df

 23%|██▎       | 48/211 [00:00<00:02, 59.59it/s]


RuntimeError: CUDA error: CUBLAS_STATUS_INTERNAL_ERROR when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`

In [ ]:
result_df